In [ ]:
import json
import os
import rasterio as rio
import rioxarray
import hvplot.xarray

In [ ]:
s3_path = 's3://snowex-data/uavsar-project/UAVSAR_images/grmesa_2020_008_274_0007d/grmesa_27416_20008-004_20013-004_0007d_s01_L090HH_01.cor.grd.tiff'

In [ ]:
%%bash 
# Set up temporary credentials for GDAL
# https://github.com/OSGeo/gdal/issues/4058
# NOTE: these last for 1 hour
aws sts assume-role-with-web-identity \
 --role-arn $AWS_ROLE_ARN \
 --role-session-name snowex \
 --web-identity-token file://$AWS_WEB_IDENTITY_TOKEN_FILE \
 --duration-seconds 3600 > /tmp/irp-cred.json

# Print out creds    
#echo AWS_REGION="us-west-2"
#echo AWS_ACCESS_KEY_ID="$(cat /tmp/irp-cred.json | jq -r ".Credentials.AccessKeyId")"
#echo AWS_SECRET_ACCESS_KEY="$(cat /tmp/irp-cred.json | jq -r ".Credentials.SecretAccessKey")"
#echo AWS_SESSION_TOKEN="$(cat /tmp/irp-cred.json | jq -r ".Credentials.SessionToken")"

In [ ]:
# Use temporary credentials in python (rasterio)

with open('/tmp/irp-cred.json') as f:
    creds = json.load(f)
    AccessKeyId = creds['Credentials']['AccessKeyId']
    SecretAccessKey = creds['Credentials']['SecretAccessKey']
    SessionToken = creds['Credentials']['SessionToken']
    
    # Set environment variables
    os.environ['AWS_ACCESS_KEY_ID'] = AccessKeyId
    os.environ['AWS_SECRET_ACCESS_KEY'] = SecretAccessKey
    os.environ['AWS_SESSION_TOKEN'] = SessionToken

# GDAL env variables https://gdal.org/user/virtual_file_systems.html#vsicurl-http-https-ftp-files-random-access
Env = rio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR')

In [ ]:
# Now you can read it directly and work with subsets, etc
import rioxarray
with Env:
    da = rioxarray.open_rasterio(s3_path, masked=True).squeeze('band', drop=True)

da.name = os.path.basename(s3_path)
da

In [ ]:
# Efficiently plot in browser
import hvplot.xarray
da.hvplot.image(rasterize=True, cmap='gray')

In [ ]:
#http://xarray.pydata.org/en/stable/user-guide/indexing.html
# Use da.sel to use geo coordinates
subset = da.isel(x=slice(1000,2000), y=slice(4000,4500))
subset